# Network in Network
- Minh Nguyen
- 1/16/2025
- Idea: replace traditional convolutional layers with "micro-networks" (MLP) to perform a more complex feature transformation
    - Instead of a single linear transformation in each convolutional layer, NiN use 1x1 convolutions (mini-MLPs) to compute more complex and non-linear feature representations, which improves the model's expressiveness.
- Key Features:
    - MLPConv Layers:
        - Each convolution is followed by a 1 x 1 convolution, treating the spatial dimensions as individual data points.
        - Allows the network to learn local interactions and channel-wise transformations.
    - Global Average Pooling:
        - Replaces fully conneted layers for classification.
        - Redues overfitting and computational cost
    - Parameter Efficiency:
        - By using 1 x 1 convolutions, NiN reduces the number of parameters compared to fully connected layer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

class NiN(nn.Module):
    def __init__(self, num_classes=10):
        super(NiN, self).__init__()
        
        # Define the NiN architecture
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 10, kernel_size=5, stride=1, padding=2),  # 1x28x28 -> 10x28x28
            nn.ReLU(inplace=True),
            nn.Conv2d(10, 10, kernel_size=1),  # MLPConv layer (10x28x28 -> 10x28x28)
            nn.ReLU(inplace=True),
            nn.Conv2d(10, 10, kernel_size=1),  # MLPConv layer (10x28x28 -> 10x28x28)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  # 10x28x28 -> 10x14x14
            
            # Block 2
            nn.Conv2d(10, 20, kernel_size=5, stride=1, padding=2),  # 10x14x14 -> 20x14x14
            nn.ReLU(inplace=True),
            nn.Conv2d(20, 20, kernel_size=1),  # MLPConv layer (20x14x14 -> 20x14x14)
            nn.ReLU(inplace=True),
            nn.Conv2d(20, 20, kernel_size=1),  # MLPConv layer (20x14x14 -> 20x14x14)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  # 20x14x14 -> 20x7x7
            
            # Block 3
            nn.Conv2d(20, 40, kernel_size=3, stride=1, padding=1),  # 20x7x7 -> 40x7x7
            nn.ReLU(inplace=True),
            nn.Conv2d(40, 40, kernel_size=1),  # MLPConv layer (40x7x7 -> 40x7x7)
            nn.ReLU(inplace=True),
            nn.Conv2d(40, 40, kernel_size=1),  # MLPConv layer (40x7x7 -> 40x7x7)
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling (40x7x7 -> 40x1x1)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(40, num_classes)
        )

    def forward(self, x):
        x = self.features(x)  # Feature extraction
        x = self.classifier(x)  # Classification
        return x

# Prepare Fashion-MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define training settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NiN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Testing Loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
